# TFIDF-ml

# 1、data extract

In [1]:
import json

In [2]:
import re
def remove_url(text):
    '''
    args:
        text: the tweet text
    return:
        new_text: tweet text removed the url
    '''
    URL_REGEX = URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:\'\".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    url_lists = re.findall(URL_REGEX,text)
    for url in url_lists:
        new_text = text.replace(url,'')
        text = new_text
    return text

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

tt = TweetTokenizer(strip_handles=True)
stopwords = set(stopwords.words('english'))

# tt = TweetTokenizer(strip_handles=True)
def lower_word(data):
    """
    args: 
        data:  text while text include source text and reply text
        
    return:
        result: text while text include source text and reply text(remove the stopwords)
    """
    res = ''
    for word in tt.tokenize(data):
        if word.lower().strip() in stopwords:
            continue
        elif word.lower().startswith("#"):
            res += ' hashtag_'+word.lower()[1:] 
        elif word.lower().startswith("@") and len(word) > 1:
            res += ' att_'+word.lower()[1:] 
        elif word.lower().startswith("@") and len(word) == 1:
            continue
        else:
            res += ' '+word.lower()
    return res

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wangyibo06/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def prepare_data(filename):
    """
    args: 
        data: {user_id:{"text":text,"user":user,"place_id":place_id}}
     
    return:
        result: {text: [text1,text2,...],length:[length1,length2,...],label:[rumor or not,...] } 
    """
    with open(filename, 'r') as obj:
        for line in obj.readlines():
            data = json.loads(line)
    result = {}
    text = []
    textlabel = []
    location = []
    length = []
    for user_id in data.keys():
        text.append(lower_word(remove_url(data[user_id]['user']['location']+" "+data[user_id]['text'])))
        textlabel.append(data[user_id]['place_id']) 
        length.append(len(lower_word(remove_url(data[user_id]['user']['location']+" "+data[user_id]['text']))))
        location.append(data[user_id]['user']['location'])
    result['text'] = text
    result['length'] = length
    result['label'] = textlabel
    result['location'] = location
    return result

In [5]:
file = './train_dev_data/0905_1005.txt'
train_dev = prepare_data(file)

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
train_dev_df = pd.DataFrame(train_dev)
train_dev_df.head()

,text,length,label,location
0,"west end , brisbane posted photo west end",42,4,"West End, Brisbane"
1,"melbourne , australia posted photo caulfield ...",1555,3,"Melbourne, Australia"
2,"sydney , new south wales watercolour art clas...",194,1,"Sydney, New South Wales"
3,"sydney , new south wales posted photo centenn...",867,1,"Sydney, New South Wales"
4,"bishopsbourne , tasmania posted photo richmon...",154,11,"Bishopsbourne, Tasmania"


In [8]:
#split the train_dev dataset
from sklearn.model_selection import train_test_split
x_processed = train_dev_df['text']
y_processed = train_dev_df['label']
x_train,x_dev,y_train,y_dev = train_test_split(x_processed,y_processed,test_size = 0.3,stratify = y_processed,random_state = 22)

In [9]:
print(len(set(x_train)))
print(len(x_train))

2078
2132


In [10]:
print(len(x_train))
print(len(x_dev))
print(y_train.head())

2132
915
121     1
1473    0
1741    4
2189    5
2666    1
Name: label, dtype: int64


In [11]:
print(len(x_train.iloc[:].values))

2132


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier, LogisticRegression
# from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

In [13]:
# from sklearn.metrics import f1_score
#lr ngram 4,features 10000
from sklearn import svm

best_score = 0
best_clf = None
best_tfidf = None
for max_n in range(2,6):
    for features in range(10000,30000,10000):
        tfidf = TfidfVectorizer(ngram_range=(1, max_n), max_features=features).fit(x_train.iloc[:].values)
        train_tfidf = tfidf.transform(x_train.iloc[:].values)
        dev_tfidf = tfidf.transform(x_dev.iloc[:].values)
        clf = LogisticRegression()
        clf.fit(train_tfidf, y_train.iloc[:].values)
        val_pred = clf.predict(dev_tfidf)
        score = f1_score(y_dev.iloc[:].values, val_pred, average='macro')
        print('max_n:',max_n,'features:',features,'score:%.4f'%score)
        if score > best_score:
            best_score = score
            best_clf = clf
            best_max_n = max_n
            best_features = features
print('best_n:',best_max_n,'best_features:',best_features,'best_score:%.4f'%best_score)


max_n: 2 features: 10000 score:0.4684
max_n: 2 features: 20000 score:0.4527
max_n: 3 features: 10000 score:0.4802
max_n: 3 features: 20000 score:0.4493
max_n: 4 features: 10000 score:0.4851
max_n: 4 features: 20000 score:0.4586
max_n: 5 features: 10000 score:0.4902
max_n: 5 features: 20000 score:0.4659
best_n: 5 best_features: 10000 best_score:0.4902


In [14]:
# from sklearn.metrics import f1_score
#lr ngram 4,features 10000
from sklearn import svm

best_score = 0
best_clf = None
best_tfidf = None
for max_n in range(5,6):
    for features in range(1000,20000,1000):
        tfidf = TfidfVectorizer(ngram_range=(1, max_n), max_features=features).fit(x_train.iloc[:].values)
        train_tfidf = tfidf.transform(x_train.iloc[:].values)
        dev_tfidf = tfidf.transform(x_dev.iloc[:].values)
        clf = LogisticRegression()
        clf.fit(train_tfidf, y_train.iloc[:].values)
        val_pred = clf.predict(dev_tfidf)
        score = f1_score(y_dev.iloc[:].values, val_pred, average='macro')
        print('max_n:',max_n,'features:',features,'score:%.4f'%score)
        if score > best_score:
            best_score = score
            best_clf = clf
            best_max_n = max_n
            best_features = features
print('best_n:',best_max_n,'best_features:',best_features,'best_score:%.4f'%best_score)


max_n: 5 features: 1000 score:0.4659
max_n: 5 features: 2000 score:0.4758
max_n: 5 features: 3000 score:0.4908
max_n: 5 features: 4000 score:0.4967
max_n: 5 features: 5000 score:0.5033
max_n: 5 features: 6000 score:0.5042
max_n: 5 features: 7000 score:0.5014
max_n: 5 features: 8000 score:0.4923
max_n: 5 features: 9000 score:0.5035
max_n: 5 features: 10000 score:0.4902
max_n: 5 features: 11000 score:0.4828
max_n: 5 features: 12000 score:0.4805
max_n: 5 features: 13000 score:0.4831
max_n: 5 features: 14000 score:0.4826
max_n: 5 features: 15000 score:0.4802
max_n: 5 features: 16000 score:0.4782
max_n: 5 features: 17000 score:0.4747
max_n: 5 features: 18000 score:0.4652
max_n: 5 features: 19000 score:0.4673
best_n: 5 best_features: 6000 best_score:0.5042


In [15]:
print('best_n:',best_max_n,'best_features:',best_features,'best_score:%.4f'%best_score)

best_n: 5 best_features: 6000 best_score:0.5042


In [39]:
import joblib
# store the model
joblib.dump(best_clf, 'tfidf-lr_ngram{}_features{}_score{}.pkl'.format(best_max_n,best_features,best_score))

['tfidf-lr_ngram5_features4000_score0.40955266149151254.pkl']

In [17]:
best_max_n = 3
best_features = 7000

In [16]:
from sklearn import metrics
import numpy as np
tfidf = TfidfVectorizer(ngram_range=(1,best_max_n+1), max_features=best_features).fit(x_train.iloc[:].values)
train_tfidf = tfidf.transform(x_train.iloc[:].values)
dev_tfidf = tfidf.transform(x_dev.iloc[:].values)

clf = LogisticRegression()
clf.fit(train_tfidf, y_train.iloc[:].values)

val_pred = clf.predict(dev_tfidf)
# score = f1_score(y_dev.iloc[:].values, val_pred, average='macro')
score = f1_score(y_dev.iloc[:].values, val_pred, average='micro')

print('best_score:%.4f'%score)
print(metrics.classification_report(y_dev.iloc[:].values,val_pred))

best_score:0.6699
              precision    recall  f1-score   support

          -1       1.00      0.10      0.18        20
           0       0.77      0.39      0.52        69
           1       0.60      0.90      0.72       214
           2       0.67      0.32      0.43        38
           3       0.71      0.87      0.78       187
           4       0.67      0.63      0.65        79
           5       0.62      0.61      0.62        95
           6       0.00      0.00      0.00        13
           7       0.73      0.65      0.69        51
           8       0.80      0.68      0.74        60
           9       1.00      0.18      0.31        22
          10       0.75      0.43      0.55         7
          11       0.80      0.36      0.50        11
          12       0.50      0.32      0.39        22
          13       1.00      0.50      0.67         6
          14       0.93      0.65      0.76        20
          15       0.00      0.00      0.00         1

    accu

/Users/wangyibo06/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wangyibo06/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wangyibo06/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
#prepare for the test data
def prepare_test(filename):
    """
    args: 
        data: {user_id:{"text":text,"user":user,"place_id":place_id}}
     
    return:
        result: {text: [text1,text2,...],length:[length1,length2,...],label:[rumor or not,...] } 
    """
    with open(filename, 'r') as obj:
        for line in obj.readlines():
            data = json.loads(line)
    result = {}
    text = []
    location = []
    length = []
    user_ids = []
    for user_id in data.keys():
        user_ids.append(user_id)
        text.append(data[user_id]['text'])
        length.append(len(data[user_id]['text']))
        location.append(data[user_id]['location'])
    result['user_id'] = user_ids
    result['text'] = text
    result['length'] = length
    result['location'] = location
    return result

In [28]:
file = './test_data/0905.txt'
test = prepare_test(file)

In [29]:
test_df = pd.DataFrame(test)
test_df.head()

,user_id,text,length,location
0,276090111,@aVoice2bHrd You know how I am 😭I cut family o...,138,West Coast
1,1499908182,@AusAndy Most people probably have one now fro...,187,"Sydney, New South Wales"
2,1952211163,@anassilvvaa Bora@anassilvvaa Se envolver dinh...,1948,Invicta
3,788532504626012160,@60Mins Will this program be replayed?Is last ...,1284,"Newcastle, New South Wales"
4,1459159765,Breaking news: emergency COVID laws applying t...,133,Fortitude Valley


In [30]:
from sklearn import metrics
import numpy as np
tfidf = TfidfVectorizer(ngram_range=(1,best_max_n+1), max_features=best_features).fit(x_train.iloc[:].values)
# train_tfidf = tfidf.transform(x_train.iloc[:].values)
# dev_tfidf = tfidf.transform(x_dev.iloc[:].values)
test_tfidf = tfidf.transform(test_df['text'].iloc[:].values)

clf = LogisticRegression()
clf.fit(train_tfidf, y_train.iloc[:].values)

val_pred = clf.predict(test_tfidf)
print(val_pred)

[1 3 1 ... 3 1 1]


In [31]:
#construct the test output file
test_df['predict_label'] = val_pred
test_df.head()

,user_id,text,length,location,predict_label
0,276090111,@aVoice2bHrd You know how I am 😭I cut family o...,138,West Coast,1
1,1499908182,@AusAndy Most people probably have one now fro...,187,"Sydney, New South Wales",3
2,1952211163,@anassilvvaa Bora@anassilvvaa Se envolver dinh...,1948,Invicta,1
3,788532504626012160,@60Mins Will this program be replayed?Is last ...,1284,"Newcastle, New South Wales",3
4,1459159765,Breaking news: emergency COVID laws applying t...,133,Fortitude Valley,1


In [35]:
## write predictions to json
# from collection import OrderedDict
def write2json(filename,dataframe):
    """
    args: 
        filename: the filename of the predicted data label file
        dataframe: the dataframe of the predicted data
    return:
        None
    """
    file_cnt = 0
    new_dict = {}
    with open(filename, 'w') as file:
        
        for index,row in dataframe.iterrows():
            new_dict['id'] = row['user_id']
            new_dict['place_id'] = row['predict_label']
        
            json_line = json.dumps(new_dict)
            file.write(json_line+'\n')
            file_cnt += 1
            if file_cnt % 100 == 0:
                print('file:'+str(file_cnt))
    return None

In [36]:
write2json('./ml_predict/lr/lr_0905_v10.json',test_df)

file:100
file:200
file:300
file:400
file:500
file:600
file:700
file:800
file:900
file:1000
file:1100
file:1200
file:1300
file:1400
file:1500
file:1600
file:1700
file:1800
file:1900
file:2000
file:2100
file:2200
file:2300
file:2400
file:2500
file:2600
file:2700
file:2800
file:2900
file:3000
file:3100
file:3200
file:3300
file:3400
file:3500
file:3600
file:3700
file:3800
file:3900
file:4000
file:4100
file:4200
file:4300
file:4400
file:4500
file:4600
file:4700
file:4800
file:4900
file:5000
file:5100
file:5200
file:5300
file:5400
file:5500
file:5600
file:5700
file:5800
file:5900
file:6000
file:6100
file:6200
file:6300
file:6400
file:6500
file:6600
file:6700
file:6800
file:6900
file:7000
file:7100
file:7200
file:7300
file:7400
file:7500
file:7600
file:7700
file:7800
file:7900
file:8000
file:8100
file:8200
file:8300
file:8400
file:8500
file:8600
file:8700
file:8800
file:8900
file:9000
file:9100
file:9200
file:9300
file:9400
file:9500
file:9600
file:9700
file:9800
file:9900
file:10000
file:101